In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from scipy.io import savemat
from scipy.signal.windows import hann, hamming

def read_mumax3_table(filename):
    """Puts the mumax3 output table in a pandas dataframe"""
    from pandas import read_table
    table = read_table(filename)
    table.columns = ' '.join(table.columns).split()[1::2]
    return table
    
def read_mumax3_ovffiles(outputdir):
    """Load all ovffiles in outputdir into a dictionary of numpy arrays
    with the ovffilename (without extension) as key"""
    from subprocess import run, PIPE, STDOUT
    from glob import glob
    from os import path
    from numpy import load
    # convert all ovf files in the output directory to numpy files
    p = run(["./mumax3-convert","-numpy",outputdir+"/*.ovf"], stdout=PIPE, stderr=STDOUT)
    if p.returncode != 0:
        print(p.stdout.decode('UTF-8'))
    # read the numpy files (the converted ovf files)
    fields = {}
    for npyfile in glob(outputdir+"/*.npy"):
        key = path.splitext(path.basename(npyfile))[0]
        fields[key] = load(npyfile)
    return fields

def build_magnetization_array(fields, pattern='m'):
    """
    Build a 5D magnetization array from MuMax3 output fields.
    
    Parameters
    ----------
    fields : dict
        Dictionary of numpy arrays from read_mumax3_ovffiles()
    pattern : str
        Filename pattern to match (default: 'm' for magnetization files)
    
    Returns
    -------
    M : ndarray
        5D array with shape (component, z, y, x, t) where component indices are:
        0: x-component, 1: y-component, 2: z-component
    timestamps : list
        List of timestamps extracted from filenames
    """
    import re
    
    # Find all matching fields and extract timestamps
    time_fields = {}
    for key, data in fields.items():
        match = re.match(rf'{pattern}(\d{{6}})$', key)
        if match:
            timestamp = int(match.group(1))
            time_fields[timestamp] = data
    
    if not time_fields:
        raise ValueError(f"No fields matching pattern '{pattern}' found")
    
    # Sort by timestamp
    timestamps = sorted(time_fields.keys())
    
    # Get dimensions from first field
    first_field = time_fields[timestamps[0]]
    ncomp, nz, ny, nx = first_field.shape
    nt = len(timestamps)
    
    if ncomp != 3:
        raise ValueError(f"Expected 3 components, got {ncomp}")
    
    # Build the 5D array
    M = np.zeros((ncomp, nz, ny, nx, nt))
    for idx, t in enumerate(timestamps):
        M[:, :, :, :, idx] = time_fields[t]
    
    return M, timestamps
    
def mag_tfft_select(M, component, dt, fsel, dimorder='zyx', detrend=True, 
                    window='hann', stat='amp'):
    """
    Extract amplitude/power maps at specified frequencies from magnetization data.
    
    Parameters
    ----------
    M : ndarray
        5D magnetization array with shape (component, z, y, x, t)
    component : int or str
        Component to analyze: 0/'x', 1/'y', or 2/'z'
    dt : float
        Time step between samples (seconds)
    fsel : array-like
        Frequencies to extract (Hz)
    dimorder : str, optional
        Spatial dimension order in output. Default 'zyx' keeps z,y,x order.
        Options: 'xyz', 'xzy', 'yxz', 'yzx', 'zxy', 'zyx'
    detrend : bool, optional
        Remove mean before FFT (default: True)
    window : str, optional
        Window function: 'hann', 'hamming', or 'none' (default: 'hann')
    stat : str, optional
        Output statistic: 'amp' (amplitude), 'power', or 'complex' (default: 'amp')
    
    Returns
    -------
    maps : list of ndarrays
        List of 3D spatial maps (one per frequency)
    fpos : ndarray
        Actual frequency bin positions (Hz)
    """
    # Parse component
    if isinstance(component, str):
        comp_map = {'x': 0, 'y': 1, 'z': 2}
        ci = comp_map.get(component.lower())
        if ci is None:
            raise ValueError("component must be 'x', 'y', 'z' or 0, 1, 2")
    elif component in [0, 1, 2]:
        ci = component
    else:
        raise ValueError("component must be 'x', 'y', 'z' or 0, 1, 2")
    
    # Extract component: M is (comp, z, y, x, t)
    ncomp, nz, ny, nx, nt = M.shape
    X = M[ci, :, :, :, :]  # (z, y, x, t)
    
    # Reorder spatial dimensions according to dimorder
    # Current order is z, y, x (dims 0, 1, 2), time is dim 3
    dimorder_map = {
        'xyz': [2, 1, 0, 3],  # x, y, z, t
        'xzy': [2, 0, 1, 3],  # x, z, y, t
        'yxz': [1, 2, 0, 3],  # y, x, z, t
        'yzx': [1, 0, 2, 3],  # y, z, x, t
        'zxy': [0, 2, 1, 3],  # z, x, y, t
        'zyx': [0, 1, 2, 3],  # z, y, x, t (no change)
    }
    
    if dimorder.lower() not in dimorder_map:
        raise ValueError(f"Invalid dimorder: {dimorder}")
    
    perm = dimorder_map[dimorder.lower()]
    X = np.transpose(X, perm)  # Reordered to (d1, d2, d3, t)
    
    nd1, nd2, nd3, nt = X.shape
    nvox = nd1 * nd2 * nd3
    
    # Reshape to (nt, nvox) for efficient FFT along time axis
    X = X.reshape(nd1 * nd2 * nd3, nt).T  # (nt, nvox)
    
    # Detrend: remove mean
    if detrend:
        X = X - np.nanmean(X, axis=0, keepdims=True)
    
    # Apply window
    if window.lower() == 'hann':
        w = hann(nt, sym=False)
    elif window.lower() == 'hamming':
        w = hamming(nt, sym=False)
    else:
        w = np.ones(nt)
    
    X = X * w[:, np.newaxis]  # Broadcast window over all voxels
    
    # FFT along time axis (axis 0)
    F = np.fft.fft(X, axis=0)
    
    # Frequency bins
    fs = 1.0 / dt
    fbin = np.fft.fftfreq(nt, dt)
    
    # Find nearest bins for requested frequencies
    fsel = np.atleast_1d(fsel)
    k = np.round(fsel / (fs / nt)).astype(int)
    k = np.clip(k, 0, nt - 1)
    fpos = k * (fs / nt)
    
    # Extract frequency slices and reshape to 3D spatial maps
    maps = []
    for ki in k:
        row = F[ki, :]  # Shape (nvox,)
        vol = row.reshape(nd1, nd2, nd3)
        
        if stat.lower() == 'complex':
            maps.append(vol)
        elif stat.lower() == 'amp':
            maps.append(np.abs(vol))
        elif stat.lower() == 'power':
            maps.append(np.abs(vol)**2)
        else:
            raise ValueError("stat must be 'complex', 'amp', or 'power'")
    
    return maps, fpos
    
def run_mumax3(script, name, verbose=True):
    """ Executes a mumax3 script and convert ovf files to numpy files
    Parameters
    ----------
      script:  string containing the mumax3 input script
      name:    name of the simulation (this will be the name of the script and output dir)
      verbose: print stdout of mumax3 when it is finished
    """
    from subprocess import run, PIPE, STDOUT
    from os import path
    scriptfile = name + ".txt"
    outputdir  = name + ".out"
    # write the input script in scriptfile
    with open(scriptfile, 'w' ) as f:
        f.write(script)
    # call mumax3 to execute this script
    p = run(["./mumax3","-f",scriptfile], stdout=PIPE, stderr=STDOUT)
    if verbose or p.returncode != 0:
        print(p.stdout.decode('UTF-8'))
    if path.exists(outputdir + "/table.txt"):
        table = read_mumax3_table(outputdir + "/table.txt")
    else:
        table = None
    fields = read_mumax3_ovffiles(outputdir)
    return table, fields

#--------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------

# Do python calculations not possible in MuMax3 here.
# -------------------------------------------------------
# Generate random dot positions in Python
np.random.seed(42)  # Use fixed seed for reproducibility
n_dots = 8
# x range: -2.4 µm to -0.1 µm (main arena, avoiding edges and junction)
# y range: -0.4 µm to +0.4 µm (80% of device height)
dot_positions = []
for i in range(n_dots):
    x_pos = -2.4e-6 + np.random.random() * 2.3e-6  # Random x between -2.4 and -0.1 µm
    y_pos = -0.4e-6 + np.random.random() * 0.8e-6  # Random y between -0.4 and +0.4 µm
    dot_positions.append((x_pos, y_pos))

print(f"Generated {n_dots} dot positions:")
for i, (x, y) in enumerate(dot_positions):
    print(f"  Dot {i+1}: x={x*1e6:.3f} µm, y={y*1e6:.3f} µm")

#PASTE THE SCRIPT HERE
script=f"""
n := {n_dots} // The number of random dots to generate

//  Set up universe & geometry ------------
// ----------------------------------------
cell_size_x := 20e-9
cell_size_y := 20e-9
cell_size_z := 10e-9
device_size_x := 5e-6
device_size_y := 1e-6
device_size_z := 100e-9
//SetGridsize(device_size_x/cell_size_x, device_size_y/cell_size_y, device_size_z/cell_size_z)
SetGridsize(250, 50, 20)
SetCellsize(cell_size_x, cell_size_y, cell_size_z)
// Main chamber now extends from left edge to center (3 µm x 1 µm)
main_arena := cuboid(3e-6, 1e-6, 100e-9).transl(-1e-6, 0, -50e-9)
output_corridor_top := (cuboid(2e-6, 0.3e-6, 100e-9).transl(1.5e-6, 0.35e-6, 0)).transl(0, 0, -50e-9)
output_corridor_bottom := (cuboid(2e-6, 0.3e-6, 100e-9).transl(1.5e-6, -0.35e-6, 0)).transl(0, 0, -50e-9)
device := (main_arena.add(output_corridor_top)).add(output_corridor_bottom)
// ----------------------------------------


// Define material parameters -------------
// ----------------------------------------
edge_alpha := 0.5 // This should be the alpha at the edges of the device.
device_alpha := 2e-4    // This is the main alpha inside the device.
soft_alpha_edges := 1   // Flag to turn on alpha modulation at device edges
Msat = 1.4e5
Aex = 3.5e-12
L_ex := sqrt(Aex.Average() / (0.5 * 4 * pi * 1e-7 * Msat.Average() * Msat.Average()))
print("Exchange Length ", L_ex)
// ----------------------------------------


// Exponentially decrease alpha inside the device ---
// ----------------------------------------
if soft_alpha_edges==1{{
    alpha = edge_alpha // Use the edge alpha by default to avoid spin wave reflections.
    delta := 250e-9        // This is the length scale of the exponential modulation for ALL walls
    k := -1e7     // Sets the rate of the exponential function.
    a := (device_alpha - edge_alpha)/(exp(k*delta) - 1)
    b := edge_alpha - a
    temp_device := device
    
    // Calculate initial device dimensions to determine scaling factors
    // Main arena: 3 µm x 1 µm, outputs extend 2 µm right
    // Total bounding box is approximately 5 µm (x) x 1 µm (y)
    Lx0 := device_size_x  // 5 µm - full x extent
    Ly0 := device_size_y  // 1 µm - full y extent
    
    for i:=1; i<254; i+=1{{ 
        // Calculate how far into the device this shell is (distance from edge)
        x := delta*i/253
        new_alpha := a*exp(k*x)+b
        //new_alpha := edge_alpha - (i/253)*(edge_alpha - device_alpha) // Uncomment this line for linear edge modifications, instead of exponential. 
        
        // Calculate independent scale factors for x and y based on edge transition width
        // Each dimension shrinks by delta/253 at each step
        scale_x := (Lx0 - 2*i*(delta/253))/(Lx0 - 2*(i-1)*(delta/253))
        scale_y := (Ly0 - 2*i*(delta/253))/(Ly0 - 2*(i-1)*(delta/253))
        
        // Apply anisotropic scaling: different factors for x and y
        temp_device = temp_device.scale(scale_x, scale_y, 1.0)
        defRegion(i,temp_device)
        alpha.setRegion(i,new_alpha)
    }}
}} else {{
    alpha = device_alpha 
}}
saveas(alpha, "alpha_map")
// ----------------------------------------


// Add magnetic dots above device (region 254)
// ----------------------------------------
dots := cylinder(0, 0)
if n != 0 {{
    // Use pre-generated dot positions from Python
{"\n".join([f"    dots = dots.add((cylinder(100e-9,100e-9).transl({x:.15e}, {y:.15e}, 0)).transl(0, 0, 50e-9))" for x, y in dot_positions])}
    defregion(254, dots)
    Msat.setRegion(254, 1.145e6)
    Aex.setRegion(254, 7.5e-12)
    alpha.setRegion(254, 0.2)
}} 
// ----------------------------------------

// Set the geometry -----------------------
// ----------------------------------------
totalgeom := device.add(dots)
setgeom(totalgeom)
saveas(geom, "sample_geometry")
// ----------------------------------------

// Simulation parameters ------------------
// ----------------------------------------
T = 50e-9         // Simulation time
f1 := 2.6e9        // Input frequency 1
f2 := 2.8e9        // Input frequency 2
sample_dt := 50e-12     // Sample time 
m = uniform(0.02, 0.02, 1)
saveas(m, "initial_m")
//relax()
B_ext = vector(0, 0, 0.2)
//saveas(m, "relaxed_m")
autosave(m,sample_dt)
autosave(B_ext,sample_dt)
// ----------------------------------------


// Input stripline (region 255) -----------
// ----------------------------------------
// Stripline at left edge, spanning 80% of y dimension (0.8 µm height)
input_stripline := cuboid(300e-9, 0.8e-6, 100e-9).transl(-2.5e-6 + 150e-9, 0, -50e-9)
defregion(255,input_stripline)
B_ext.setregion(255, vector(0, (0.1e-3)*sin(2*pi*f1*t) + (0.1e-3)*sin(2*pi*f2*t), 0.2))
saveas(regions, "regions_map")
// ----------------------------------------


// Run ------------------------------------
// ----------------------------------------
run(T)
// ----------------------------------------
"""
# Run the script in MuMax3 and save the.ovf files as .mat
table, fields = run_mumax3(script, name="spinwave_device_dots", verbose=True )
savemat("spinwave_device_dots.mat", fields, appendmat=True) # Output all .ovf files to .mat for MATLAB

In [ ]:
# Process simulation results with FFT analysis
# ---------------------------------------------

# Build 5D magnetization array from OVF files
M, timestamps = build_magnetization_array(fields, pattern='m')
print(f"Magnetization array shape: {M.shape}")
print(f"(component, z, y, x, time) = ({M.shape[0]}, {M.shape[1]}, {M.shape[2]}, {M.shape[3]}, {M.shape[4]})")
print(f"Number of timesteps: {len(timestamps)}")

# FFT analysis parameters
dt = 50e-12  # 50 ps sampling time from simulation
freqs = [2.6e9, 2.8e9]  # Input frequencies

# Extract spectral amplitude maps for z-component at both frequencies
maps, fpos = mag_tfft_select(M, component='z', dt=dt, fsel=freqs, 
                             dimorder='zyx', detrend=True, 
                             window='hann', stat='amp')

print(f"\nExtracted {len(maps)} frequency maps")
print(f"Requested frequencies: {freqs}")
print(f"Actual bin frequencies: {fpos}")

# Simulation dimensions (from script)
device_size_x = 5e-6  # 5 µm
device_size_y = 1e-6  # 1 µm
cell_size_x = 20e-9
cell_size_y = 20e-9
nx, ny = 250, 50

# Calculate aspect ratio and extent
aspect_ratio = device_size_y / device_size_x  # height / width
extent = [-device_size_x/2 * 1e6, device_size_x/2 * 1e6,  # x in µm
          -device_size_y/2 * 1e6, device_size_y/2 * 1e6]  # y in µm

# Visualize results - use larger figure size
fig_width = 10 * len(maps)  # 10 inches per subplot
fig_height = fig_width * aspect_ratio  # Maintain true aspect ratio
fig, axes = plt.subplots(1, len(maps), figsize=(fig_width, fig_height))
if len(maps) == 1:
    axes = [axes]

for i, (fmap, f) in enumerate(zip(maps, fpos)):
    # Sum over z-axis to get 2D projection
    map_2d = np.sum(np.abs(fmap), axis=0)
    
    # Plot with correct aspect ratio and physical dimensions
    im = axes[i].imshow(map_2d, origin='lower', aspect='equal', cmap='turbo',
                        extent=extent, interpolation='bilinear')
    
    # Overlay dot positions using known coordinates from Python
    from matplotlib import patches
    for x_pos, y_pos in dot_positions:
        # Draw circle at each dot position (100 nm diameter = 0.1 µm radius)
        circle = patches.Circle((x_pos*1e6, y_pos*1e6), 0.05, 
                               fill=False, edgecolor='white', 
                               linewidth=1.5, alpha=0.8)
        axes[i].add_patch(circle)
    
    axes[i].set_title(f'f = {f/1e9:.2f} GHz')
    axes[i].set_xlabel('x (µm)')
    axes[i].set_ylabel('y (µm)')
    plt.colorbar(im, ax=axes[i], label='Amplitude (a.u.)', fraction=0.046, pad=0.04)

plt.tight_layout()
plt.savefig('spectral_maps.png', dpi=150, bbox_inches='tight')
plt.show()